In [5]:
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import import_ipynb
import feature_engineering

import surprise

importing Jupyter notebook from feature_engineering.ipynb
importing Jupyter notebook from data_acquisition.ipynb


In [6]:
ratings = pickle.load(open("data/ratings_feature_set.pkl", "rb"))
ratings_train, ratings_test = feature_engineering.train_test_split(ratings, size = 0.1)

In [8]:
# def baseline_model():
results = BaselineOnly()

NameError: name 'BaselineOnly' is not defined

In [ ]:
ratings_train, ratings_test = feature_engineering.train_test_split(ratings, size = 0.1)